In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [8]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
from numpy import loadtxt
import time
import os
from PIL import Image
from matplotlib import pyplot
import pandas as pd
import numpy as np
import tensorflow as tf
import csv

from google.colab import drive
drive.mount('/content/drive')

img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100
accuracy_average = 0
def build_generator():
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))
    return model

def build_discriminator():
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu', padding='same', 
                  input_shape=img_shape))
    model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu', padding='same'))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', 
                    input_shape=img_shape))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))

    model.summary()
    img = Input(shape=img_shape)
    validity = model(img)
    return model

optimizer = Adam(0.0002, 0.5)
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

generator = build_generator()
z = Input(shape=(100,))
img = generator(z)

discriminator.trainable = False
valid = discriminator(img)
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

def train(epochs, batch_size, save_interval):
    read=pd.read_csv("/content/drive/My Drive/Internship 2K20/csv_28x28/002aa.csv")
    read1 = read.iloc[:,:].values
    arr=np.array
    arr = read1.reshape(789,28,28)
    
    X_train=arr

    # Rescale -1 to 1
    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        global accuracy_average
        # Select a random real images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_imgs = X_train[idx]

        # Sample noise and generate a batch of fake images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_imgs = generator.predict(noise)

        # Train the discriminator
        D_loss_real = discriminator.train_on_batch(real_imgs, valid)
        D_loss_fake = discriminator.train_on_batch(fake_imgs, fake)
        D_loss = 0.5 * np.add(D_loss_real, D_loss_fake)
        
        # Train the generator
        g_loss = combined.train_on_batch(noise, valid)

        # If at save interval
        if epoch % save_interval == 0:
            # Print the progress
            print('>Accuracy real: %.0f%%, fake: %.0f%%' % (((D_loss_real[1]*0.5)/D_loss[1])*100, ((D_loss_fake[1]*0.5)/D_loss[1])*100))
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, D_loss[0], 100 * D_loss[1], g_loss))
            accuracy_average = accuracy_average + (100*D_loss[1])
            # Save generated image samples
            save_imgs(epoch)

def save_imgs(epoch):
    r, c = 5,5
    noise = np.random.normal(0, 1, (r*c, latent_dim))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5
    
    fig, axs = plt.subplots(r,c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0] , cmap='gray')
            axs[i, j].axis('off')
            fig.canvas.flush_events()
            cnt += 1

    fig.savefig("drive/My Drive/Image_36/image_%d.png" % epoch)
    plt.close()

start = time.time()
train(epochs=5000, batch_size=32, save_interval=200)
accuracy_average = accuracy_average/25
print("Accuracy: ",accuracy_average)
end = time.time()
elapsed_train_time = 'elapsed training time: {} min, {} sec '.format(int((end - start) / 60),int((end - start) % 60))
print(elapsed_train_time)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 28, 28, 32)        25632     
_________________________________________________________________
dropout_16 (Dropout)         (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
dropout_17 (Dropout)         (None, 28, 28, 32)        0

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>Accuracy real: 100%, fake: 0%
0 [D loss: 0.793732, acc.: 40.62%] [G loss: 0.268938]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>Accuracy real: 52%, fake: 48%
200 [D loss: 0.163233, acc.: 95.31%] [G loss: 4.521430]
>Accuracy real: 46%, fake: 54%
400 [D loss: 0.428718, acc.: 84.38%] [G loss: 2.264155]
>Accuracy real: 49%, fake: 51%
600 [D loss: 0.372073, acc.: 85.94%] [G loss: 2.263828]
>Accuracy real: 47%, fake: 53%
800 [D loss: 0.228417, acc.: 92.19%] [G loss: 2.252460]
>Accuracy real: 40%, fake: 60%
1000 [D loss: 0.449163, acc.: 75.00%] [G loss: 2.357227]
>Accuracy real: 50%, fake: 50%
1200 [D loss: 0.147991, acc.: 93.75%] [G loss: 3.265070]
>Accuracy real: 44%, fake: 56%
1400 [D loss: 0.344694, acc.: 84.38%] [G loss: 2.537999]
>Accuracy real: 47%, fake: 53%
1600 [D loss: 0.195931, acc.: 90.62%] [G loss: 2.752688]
>Accuracy real: 50%, fake: 50%
1800 [D loss: 0.108783, acc.: 96.88%] [G loss: 3.603648]
>Accuracy real: 51%, fake: 49%
2000 [D loss: 0.122089, acc.: 95.31%] [G loss: 3.826437]
>Accuracy real: 49%, fake: 51%
2200 [D loss: 0.080725, acc.: 98.44%] [G loss: 3.811103]
>Accuracy real: 48%, fake: 52%
2400 